In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    
    from elasticsearch import helpers
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

Loaded  .. . . . . . . .


In [2]:
df = pd.read_excel("elastic_Search_Data.xlsx")
print("read")

read


In [3]:
df.head()

,filename,country,Product name,Refined Product Name,para_text,Possible Questions,Comment
0,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is indicated f...,What is ATACAND used for\nWhat is ATACAND used...,NaN
1,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Geriatrics (> 65 years of age) : No overall di...,What is the efficacy of ATACAND in elderly pop...,NaN
2,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Pediatrics (6 to 17 years of age): \n•\tHypert...,What is ATACAND used for in children\nWhat is ...,NaN
3,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is contraindic...,Who is ATACAND conracindicated in\nWho is ATAC...,NaN
4,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),There is evidence that co-administration of an...,Can ATACAND be co-administered with aliskiren\...,NaN


In [8]:
from bert_serving.client import BertClient
bc = BertClient()

In [9]:
answers=list(df["para_text"])

In [10]:
answers

['ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'Geriatrics (> 65 years of age) : No overall differences in safety or effectiveness were observed between these subjects and younger subjects, and other reported clinical experience has not identified differences in responses between the elderly and younger patients, but greater sensitivity of some older individuals cannot be ruled out. ',
 'Pediatrics (6 to 17 years of age): \n•\tHypertension\nATACAND is indicated for the treatment of essential hypertension in children and adolescents 6 to 17 years of age (se

In [11]:
bc.encode(answers)

array([[-0.21258351, -0.09253826, -0.0774115 , ...,  0.1889648 ,
        -0.20395528,  0.00088361],
       [-0.17276913, -0.36519182, -0.11895196, ..., -0.12006999,
         0.10894845,  0.00404331],
       [-0.27216518, -0.07298212, -0.04816346, ...,  0.12562566,
         0.15384376,  0.12629065],
       ...,
       [-0.09729759,  0.15539922,  0.02649372, ..., -0.0783862 ,
         0.07789744,  0.42761406],
       [-0.08624227, -0.06090786,  0.04918772, ..., -0.00910199,
         0.27791578, -0.07775551],
       [ 0.00739391,  0.07879533,  0.02765172, ...,  0.05927157,
        -0.04922827,  0.01016213]], dtype=float32)

In [16]:
ml_vec=bc.encode(answers)

In [18]:
type(ml_vec[0])

numpy.ndarray

In [19]:
len(ml_vec[0])

768

In [20]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "ml_vector":{
         "type":"dense_vector",
         "dims":768
      } 
    }
   }
}

In [21]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

python client created


In [22]:
#creating the database
IndexName = 'thisisit'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [23]:
my

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'thisisit'}

In [24]:
df.columns

Index(['filename', 'country', 'Product name', 'Refined Product Name',
       'para_text', 'Possible Questions', 'Comment'],
      dtype='object')

In [25]:
df22 = df.to_dict('records')
df22

[{'filename': 'Candesartan_PM_Reviewed.docx',
  'country': 'Canada',
  'Product name': 'Candesartan cilexetil',
  'Refined Product Name': 'Atacand (Candesartan Cilexetil)',
  'para_text': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'Possible Questions': 'What is ATACAND used for\nWhat is ATACAND used for in Canada\nHow can ATACAND be used\nHow can ATACAND be used in Canada\nCan ATACAND be used with calcium channel blockers\nCan ATACAND be used with calcium channel blockers in Canada\nCan ATACAND (candesartan cilexetil) be indicated for heartfailure',
  

In [26]:
len(df22)

99

In [27]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("para_text",""),
    'ml_vector':ml_vec[c]})

In [28]:
arr

[{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'ml_vector': array([-2.12583512e-01, -9.25382599e-02, -7.74115026e-02,  2.08734185e-01,
          2.33869508e-01, -1.54881433e-01, -1.55370027e-01, -2.10732982e-01,
          2.48278528e-02, -1.23438403e-01,  2.50698566e-01, -4.60335100e-03,
          2.35185493e-02, -2.72644162e-01,  3.34664047e-01, -4.34827507e-02,
         -2.80494001e-02, -3.24270129e-01, -2.32795045e-01,  2.65478194e-01,
          2.06407860e-01, -5.23196608e-02,  1.10645983e-02, -2.33840197e-02,
         -4.47063707e-02, -1.6

In [29]:
arr[0]

{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'ml_vector': array([-2.12583512e-01, -9.25382599e-02, -7.74115026e-02,  2.08734185e-01,
         2.33869508e-01, -1.54881433e-01, -1.55370027e-01, -2.10732982e-01,
         2.48278528e-02, -1.23438403e-01,  2.50698566e-01, -4.60335100e-03,
         2.35185493e-02, -2.72644162e-01,  3.34664047e-01, -4.34827507e-02,
        -2.80494001e-02, -3.24270129e-01, -2.32795045e-01,  2.65478194e-01,
         2.06407860e-01, -5.23196608e-02,  1.10645983e-02, -2.33840197e-02,
        -4.47063707e-02, -1.69696018e

In [30]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [31]:
resp

{'_index': 'thisisit',
 '_type': '_doc',
 '_id': '98',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 98,
 '_primary_term': 1}

In [33]:
#testing
que = "what is ATACAND?"
que_vec=bc.encode([que])

In [34]:
que_vec

array([[ 2.37761244e-01,  2.06043720e-02,  3.80197652e-02,
         1.95693851e-01, -2.32234776e-01, -1.43766150e-01,
         5.35774589e-01,  1.81545451e-01,  3.31313819e-01,
        -4.79843020e-01,  1.10786790e-02,  4.28289734e-02,
        -6.09774403e-02, -1.12959430e-01,  6.04255274e-02,
         8.23888481e-02,  2.85537448e-02, -1.91068545e-03,
        -4.10824925e-01,  6.55906275e-03,  3.20410542e-02,
         1.30804643e-01, -8.43904763e-02, -4.80302833e-02,
         7.34819993e-02,  5.25959395e-02, -6.53040558e-02,
        -1.69359893e-01,  1.80883721e-01,  3.71789545e-01,
         7.74598792e-02,  1.72599360e-01, -3.90767664e-01,
        -3.57651740e-01,  1.73365906e-01, -2.66161293e-01,
         1.52327437e-02, -4.92448539e-01, -1.91211268e-01,
        -1.58721000e-01,  1.18021294e-01,  3.52580845e-01,
         2.05896169e-01, -2.06625283e-01,  6.11710623e-02,
         5.89119315e-01, -1.85396113e-02, -3.87728512e-01,
         1.80392861e-01,  2.64162242e-01,  3.34093601e-0

In [2]:

script_query = {
"script_score": {
"query": {"match_all":{}},
"script": {
"source": "cosineSimilarity(params.query_vector, doc['ml_vector']) ",
"params": {"query_vector": que_vec[0] }
}}}

NameError: name 'que_vec' is not defined

In [38]:
es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

/home/srinivas/anaconda3/envs/success/lib/python3.6/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'took': 136,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 99, 'relation': 'eq'},
  'max_score': 0.89951044,
  'hits': [{'_index': 'thisisit',
    '_type': '_doc',
    '_id': '15',
    '_score': 0.89951044,
    '_source': {'answer': 'Use of ATACAND should include appropriate assessment of renal function. '}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '62',
    '_score': 0.89938337,
    '_source': {'answer': 'ATACAND may be taken with or without food '}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '82',
    '_score': 0.8980178,
    '_source': {'answer': 'A double dose of ATACAND should never be taken to make up for a missed dose. '}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '30',
    '_score': 0.8943118,
    '_source': {'answer': 'Type 1 diabetes : There is no experience regarding the administration of ATACAND in children aged 6 to <17 years with type 1 diabetes. '